# ყველა ფაილის მისამართის ამოღება

In [67]:
!pip install librosa
!pip install pydub

In [1]:
import glob
import os
from pydub import AudioSegment

# audio ფაილების გადაყვანა wav-ში

In [2]:
def convert_to_wav(files):
    for src in files:
        dst = src[:-3] + "wav"
        s = AudioSegment.from_file(src)
        s.export(dst, format="wav")

In [3]:
folders = os.listdir('data')
for sub_dir in folders:
#     print(sub_dir)
    mp3_files = glob.glob(os.getcwd() + '/data/' + '2' + '/*.mp3')
    convert_to_wav(mp3_files)
#     m4a_files = glob.glob(os.getcwd() + '/data/' + '2' + '/*.m4a')
#     mp4_files = glob.glob(os.getcwd() + '/data/' + '2' + '/*.mp4')
#     aac_files = glob.glob(os.getcwd() + '/data/' + '2' + '/*.aac')
#     files = mp3_files + m4a_files + mp4_files + aac_files
#     რატომღაც ამას არ შვება :(
#     convert_to_wav(files) 

In [2]:
import pandas as pd
import librosa
import librosa.display

##  <font color =green>extract_features ვარგა. ქვემოთ რაც წერია ყველაფერს სწორად შვება</font>

In [79]:
import numpy as np

def get_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        mfccs = get_mfcc(audio, sample_rate)
        zero_crossing_rate = get_zero_crossing_rate(audio)
#         other features
        return ([mfccs, zero_crossing_rate], [mfccs.shape, zero_crossing_rate.shape])
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 

In [63]:
def get_all_features():
    folders = os.listdir('data')
    folders.sort()
    all_features = []
    feature_sizes = []
    for label, sub_dir in enumerate(folders): #label {0; 4}
        for file_name in glob.glob(os.getcwd() + '/data/' + sub_dir + '/*.wav'):
#             print("Extracting file ", file_name)
            try:
                features, feature_size = get_features(file_name)
            except Exception as e:
                print("Extraction error")
                continue
            all_features.append(features +  [label + 1])
            feature_sizes.append(feature_size + [label + 1])
    data = pd.DataFrame(all_features, columns=['mfccs', 'zero_crossing_rate' , 'class_label'])
    data2 = pd.DataFrame(feature_sizes, columns = ['mfccs_size', 'zero_size', 'class_label'])
    return (data, data2)

In [89]:
def get_mfcc(audio, sr):
    #Mel Frequency Cepstral Coefficient
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
    mfccsscaled = np.mean(mfccs,axis=0)
#     mfccs - (40, 61), mfccsscaled - (40,)
    mfccsscaled.shape = (mfccsscaled.shape[0], 1)
    return mfccsscaled.T

In [91]:
def get_zero_crossing_rate(audio):
    #ZERO_CROSSING_RATE
    rate = librosa.feature.zero_crossing_rate(audio)
#     rate - (1, 61)
    return rate

In [16]:
def get_spectral_centroid(audio):
    #SPECTRAL_CENTROID
    sp = librosa.feature.spectral_centroid(audio)
#     sp - (1, 61)
    return sp

In [17]:
def get_spectral_roll_ff(audio, sr):
    # Spectral-roll off
    S, phase = librosa.magphase(librosa.stft(audio))
    a = librosa.feature.spectral_rolloff(S=S, sr=sr)
#     a - (1, 61)
    return a

In [18]:
def get_spectral_flux(audio, sr):
    # Spectral flux
    onset_env = librosa.onset.onset_strength(y=audio, sr=sr)
#     onset_env - (61,)
    return onset_env

In [19]:
def get_chroma_features(audio, sr):    
    # Chroma features (needs scaling)
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr) #N1
    chroma_cens = librosa.feature.chroma_cens(y=audio, sr=sr) #Chroma energy normalized statistics
    chroma_cq = librosa.feature.chroma_cqt(y=audio, sr=sr)
#     samive - (12, 61)
    return [chroma_stft, chroma_cens, chroma_cq]

In [20]:
def get_pitch(audio, sr):
    # Pitch (needs scaling)
    pitches, magnitudes = librosa.piptrack(y=audio, sr=sr)
#     pitches - (1025, 61)
    return pitches

In [92]:
data, data2 = get_all_features()
print(data2)

    mfccs_size zero_size  class_label
0      (1, 73)   (1, 73)            1
1     (1, 124)  (1, 124)            1
2     (1, 143)  (1, 143)            1
3      (1, 66)   (1, 66)            1
4     (1, 103)  (1, 103)            1
..         ...       ...          ...
161    (1, 66)   (1, 66)            5
162   (1, 121)  (1, 121)            5
163   (1, 116)  (1, 116)            5
164    (1, 70)   (1, 70)            5
165    (1, 96)   (1, 96)            5

[166 rows x 3 columns]


In [93]:
labels = [[i] for i in data['class_label']]
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(labels)
print(y_onehot.shape)

(166, 5)


/home/tamar/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [94]:
X = data.drop(columns='class_label')
print(X.shape)

(166, 2)


 # ნეირონული ქსელის კოდი

In [156]:
def sigmoid(z):
    for i in range(z.shape[0]):
        for j in range(z.shape[1]):
            x = np.exp(-z[i, j])
            z[i, j] = 1 / (1 + x)
    return z

In [157]:
def forward_propagate(X, theta1, theta2):
    #X(166,2); theta1(25, 3); theta2(5, 26)
    m = X.shape[0] #166
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1) #(166, 3)
    z2 = a1 * theta1.T #(166, 25)
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h

In [158]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return J

In [159]:
# initial setup
input_size = X.shape[1]
hidden_size = 25
num_labels = y_onehot.shape[1]
learning_rate = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y_onehot)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape

((25, 3), (5, 26))

In [160]:
a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((166, 3), (166, 25), (166, 26), (166, 5), (166, 5))

In [65]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [68]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [67]:
J, grad = backprop(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)
J, grad.shape

ValueError: shapes (166,3) and (401,25) not aligned: 3 (dim 1) != 401 (dim 0)

In [71]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, learning_rate), 
                method='TNC', jac=True, options={'maxiter': 250})
fmin

ValueError: shapes (166,3) and (401,25) not aligned: 3 (dim 1) != 401 (dim 0)

In [72]:
X = np.matrix(X)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
y_pred = np.array(np.argmax(h, axis=1) + 1)
y_pred

NameError: name 'fmin' is not defined

In [141]:
a = np.matrix([[1, 2]])
print(a.shape)
1 / (1 + np.exp(-a))

(1, 2)


matrix([[0.73105858, 0.88079708]])